In [1]:
!ls /content/drive

ls: cannot access '/content/drive': No such file or directory


In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import tensorflow as tf
import time
from tensorflow import one_hot
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import scipy
from sklearn import preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.regularizers import l2
np.random.seed(1)

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls /content/drive/My\ Drive

 Brad_work.ipynb		    hw2
'Colab Notebooks'		    Individual_Component.zip
 COMP9021_Lec1_wyn_note.ipynb	    test.csv
'Copy of new.drawio'		    training.csv
'Copy of Untitled Diagram.drawio'   Untitled0.ipynb
'CSESoc FYG 2019.gdoc'		   'Untitled Diagram (1).drawio'
'CSESoc FYG 2019.pdf'		   'Untitled Diagram.drawio'
 Group_Component.zip		   'Untitled document.gdoc'
 GSOE9820.ipynb


In [0]:
test_df = pd.read_csv("/content/drive/My Drive/test.csv")

In [6]:
test_df

,article_number,article_words,topic
0,9501,"world,complet,pharmaceut,tianjin,tianjin,chin,...",IRRELEVANT
1,9502,"copy,sunday,weekend,ec,friday,eu,includ,limit,...",IRRELEVANT
2,9503,"heavy,heavy,gabriel,morn,morn,equit,cent,cent,...",FOREX MARKETS
3,9504,"research,jess,hit,anticip,comput,comput,comput...",IRRELEVANT
4,9505,"provid,provid,luxembourg,court,court,case,opin...",IRRELEVANT
...,...,...,...
495,9996,"symantec,soon,interfac,provid,provid,comput,co...",IRRELEVANT
496,9997,"hit,hit,hit,hit,hit,hit,hit,hit,hit,beat,beat,...",SPORTS
497,9998,"cent,cent,cent,cent,match,declin,act,rate,rate...",MONEY MARKETS
498,9999,"cnmv,cnmv,stock,count,count,week,group,friday,...",SHARE LISTINGS


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/training.csv")

In [8]:
train_df

,article_number,article_words,topic
0,1,"open,absent,cent,cent,cent,stock,inflow,rate,k...",FOREX MARKETS
1,2,"morn,stead,end,end,day,day,day,patch,patch,pat...",MONEY MARKETS
2,3,"socc,socc,world,world,recent,law,fifa,fifa,fif...",SPORTS
3,4,"open,forint,forint,forint,forint,cent,cent,ste...",FOREX MARKETS
4,5,"morn,complet,weekend,minut,minut,minut,arrow,d...",IRRELEVANT
...,...,...,...
9495,9496,"cloud,provid,hope,centur,erupt,rule,recent,sou...",DEFENCE
9496,9497,"stock,stock,stock,declin,access,week,worry,blo...",IRRELEVANT
9497,9498,"rate,million,million,belarus,dollar,dollar,nov...",FOREX MARKETS
9498,9499,"flow,bullet,bullet,bullet,bullet,bullet,bullet...",IRRELEVANT


# Preprocessing
*base on the work of Raymond*

Use  ``` CountVectorizer ``` and ``` TfidfVectorizer ``` to extract features


In [9]:
train_x = train_df.article_words
train_y = train_df.topic
test_x = test_df.article_words
test_y = test_df.topic

en_train_y = preprocessing.LabelEncoder().fit_transform(train_y)
en_test_y = preprocessing.LabelEncoder().fit_transform(test_y)
categories_list=list(set(train_df['topic']))
print(categories_list)

['ARTS CULTURE ENTERTAINMENT', 'MONEY MARKETS', 'IRRELEVANT', 'BIOGRAPHIES PERSONALITIES PEOPLE', 'SPORTS', 'SHARE LISTINGS', 'HEALTH', 'DEFENCE', 'DOMESTIC MARKETS', 'SCIENCE AND TECHNOLOGY', 'FOREX MARKETS']


In [0]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    sp = tf.SparseTensor(indices, coo.data, coo.shape)
    sp = tf.sparse.reorder(sp)
    return sp

In [0]:
# get Count vectors
vectorizer = CountVectorizer().fit(train_x)
train_count_csr = vectorizer.transform(train_x)
test_count_csr = vectorizer.transform(test_x)

# change count vectors from scipy.sparse.csr.csr_matrix to tf.sparse.SparseTensor
train_count_st = convert_sparse_matrix_to_sparse_tensor(train_count_csr)
test_count_st = convert_sparse_matrix_to_sparse_tensor(test_count_csr)

# change count vectors from tf.sparse.SparseTensor to tf.Tensor
train_count_dense = tf.sparse.to_dense(train_count_st)
test_count_dense = tf.sparse.to_dense(test_count_st)

# get Tfid vectors
vectorizer = TfidfVectorizer().fit(train_x)
train_tfid_csr = vectorizer.transform(train_x)
test_tfid_csr = vectorizer.transform(test_x)

# change Tfid vectors from scipy.sparse.csr.csr_matrix to tf.sparse.SparseTensor
train_tfid_st = convert_sparse_matrix_to_sparse_tensor(train_tfid_csr)
test_tfid_st = convert_sparse_matrix_to_sparse_tensor(test_tfid_csr)

# change Tfid vecotors from tf.sparse.SparseTensor to tf.Tensor
train_tfid_dense = tf.sparse.to_dense(train_tfid_st)
test_tfid_dense = tf.sparse.to_dense(test_tfid_st)

# # get Tfid vector vocab size
# vocab_size_tfid = train_vectors.shape[1]
# print(train_vectors.shape,test_vectors.shape)

## Preprocessing data and preserve order

From ```Tokenizer``` to ```pad_sequences```

In [0]:
word_num = 500
tokenizer = Tokenizer(num_words=word_num)
tokenizer.fit_on_texts(train_x)
train_token = tokenizer.texts_to_sequences(train_x)
test_token = tokenizer.texts_to_sequences(test_x)
train_pad = pad_sequences(train_token)
word_length = train_pad.shape[1]
test_pad = pad_sequences(test_token, maxlen=word_length)

In [0]:
def cate_accuracies(matrix, categories):
    for i_row in range(len(matrix)):
        acc = matrix[i_row][i_row]/sum(matrix[i_row])
        print(f"The accuracy for {categories[i_row]} is {acc}.")

# Training Model 1

# Best Training model so far
Already tried:
* other linear layers combinations 
    * with/without dropout layers
    * with/without weight decay
* tokenize + embedding + LSTM
    * extremely slow and inaccurate, not sure if it is because of underfitting or context information not preserved since it is too slow


In [35]:
def network():
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=network, epochs=50, batch_size=20, verbose=0)

start_time = time.time()
estimator.fit(train_count_st, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
train_pred = estimator.predict(train_count_st)
print("Training confusion matrix")
print(metrics.confusion_matrix(train_y, train_pred, categories_list))

train_f1 = metrics.f1_score(train_y, train_pred, average='macro')
print(f"Training f1 score is {train_f1}.")

train_acc = metrics.accuracy_score(train_y, train_pred)
print(f"Training accuracy score is {train_acc}.")

train_recall = metrics.recall_score(train_y, train_pred, average="macro")
print(f"Training recall score is {train_recall}")

test_pred = estimator.predict(test_count_st)
print("Test confusion matrix")

confusion = metrics.confusion_matrix(test_y, test_pred, categories_list)
print(confusion)

test_f1 = metrics.f1_score(test_y, test_pred, average='macro')
print(f"Test f1 score is {test_f1}.")

test_acc = metrics.accuracy_score(test_y, test_pred)
print(f"Test accuracy score is {test_acc}.")

test_recall = metrics.recall_score(test_y, test_pred, average="macro")
print(f"Test recall score is {test_recall}")

test_report = metrics.classification_report(test_y, test_pred)
print(test_report)

cate_accuracies(confusion, categories_list)

Training time 55s
Training confusion matrix
[[   3    0   66   39    6    0    0    0    0    3    0]
 [   0 1593    9    0    0    0    0    0    0    0   71]
 [   0   50 4655    0   18    0    0    2    0    0    9]
 [   2    0    9  154    2    0    0    0    0    0    0]
 [   0    0    1    1 1100    0    0    0    0    0    0]
 [   0    0  216    0    2    0    0    0    0    0    0]
 [   3    0  143    9    0    0    0    2    0   26    0]
 [   0    0    2    0    0    0    0  256    0    0    0]
 [   0    0    5    0    0    0    0    0  128    0    0]
 [   2    0    5    0    1    0    0    1    0   61    0]
 [   0   64    4    0    0    0    0    0    0    0  777]]
Training f1 score is 0.6724143189983453.
Training accuracy score is 0.9186315789473685.
Training recall score is 0.693371366089414
Test confusion matrix
[[  0   0   3   0   0   0   0   0   0   0   0]
 [  0  43   6   0   0   0   0   0   0   0  20]
 [  1  16 240   1   3   0   0   2   1   0   2]
 [  0   0   8   7   0  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
def network():
    model = Sequential()
    model.add(Dense(128, activation='relu',
              kernel_regularizer=l2(0.01),
              bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu',
              kernel_regularizer=l2(0.01),
              bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='relu',
              kernel_regularizer=l2(0.01),
              bias_regularizer=l2(0.01)))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


estimator = KerasClassifier(build_fn=network, epochs=10, batch_size=20, verbose=0)

start_time = time.time()
estimator.fit(train_count_st, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
train_pred = estimator.predict(train_count_st)
print("Training confusion matrix")
print(metrics.confusion_matrix(train_y, train_pred, categories_list))

train_f1 = metrics.f1_score(train_y, train_pred, average='macro')
print(f"Training f1 score is {train_f1}.")

train_acc = metrics.accuracy_score(train_y, train_pred)
print(f"Training accuracy score is {train_acc}.")

train_recall = metrics.recall_score(train_y, train_pred, average="macro")
print(f"Training recall score is {train_recall}")

test_pred = estimator.predict(test_count_st)
print("Test confusion matrix")

confusion = metrics.confusion_matrix(test_y, test_pred, categories_list)
print(confusion)

test_f1 = metrics.f1_score(test_y, test_pred, average='macro')
print(f"Test f1 score is {test_f1}.")

test_acc = metrics.accuracy_score(test_y, test_pred)
print(f"Test accuracy score is {test_acc}.")

test_recall = metrics.recall_score(test_y, test_pred, average="macro")
print(f"Test recall score is {test_recall}")

test_report = metrics.classification_report(test_y, test_pred)
print(test_report)

cate_accuracies(confusion, categories_list)

Training time 14s
Training confusion matrix
[[   4    0   43   69    1    0    0    0    0    0    0]
 [   0 1594   78    0    0    0    0    0    0    0    1]
 [   1  279 4331   14   48   16   15   22    8    0    0]
 [   0    0   67   93    3    0    2    2    0    0    0]
 [   0    1    9    4 1088    0    0    0    0    0    0]
 [   0    0  103    0    1  114    0    0    0    0    0]
 [   0    0   82    2    1    0   97    1    0    0    0]
 [   0    0   72    0    0    0    1  185    0    0    0]
 [   0    0   91    0    0    0    0    0   42    0    0]
 [   0    1   23    0    1    0   20    0    0   25    0]
 [   0  800   43    0    0    0    0    0    0    0    2]]
Training f1 score is 0.5674625596731576.
Training accuracy score is 0.7973684210526316.
Training recall score is 0.5355785957701877
Test confusion matrix
[[  0   0   2   1   0   0   0   0   0   0   0]
 [  0  63   6   0   0   0   0   0   0   0   0]
 [  0  21 238   1   3   2   1   0   0   0   0]
 [  0   0   9   6   0 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
